This notebook uses `scholarly` API (https://github.com/scholarly-python-package/scholarly) to enrich CS faculty info

In [1]:
from scrap_cs_faculty import *

In [2]:
org_list = ["Princeton-CS","UWash-CS", "UPenn-CS"]
idx_org = org_list.index("Princeton-CS")
org = org_list[idx_org]
org2 = org.split("-")[0] 

## read CS faculty info scraped previously

In [3]:
file_xlsx = f"faculty-{org}.xlsx"
xlsxf = pd.ExcelFile(file_xlsx)

In [4]:
xlsxf.sheet_names

['Faculty']

In [5]:
df = xlsxf.parse('Faculty')

In [6]:
df

,name,job_title,phd_univ,phd_year,research_area,research_concentration,research_focus,url,img_url,phone,email,cell_phone,office_address,department,school,url_profile,url_author
0,Parastoo Abtahi,Assistant Professor,Stanford University,2022.0,"Human-Computer Interaction, Augmented Reality,...",NaN,NaN,https://www.cs.princeton.edu/~abtahi,https://www.cs.princeton.edu/sites/all/modules...,(609) 258-9528,abtahi@cs.princeton.edu,NaN,419 Computer Science,Computer Science,Princeton Univ,https://www.cs.princeton.edu/people/profile/ab...,NaN
1,Ryan Adams,"Professor, Associate Chair",University of Cambridge,2009.0,"I am interested in machine learning, artificia...",NaN,NaN,https://www.cs.princeton.edu/~rpa,https://www.cs.princeton.edu/sites/all/modules...,(609) 258-8682,rpa@cs.princeton.edu,NaN,411 Computer Science,Computer Science,Princeton Univ,https://www.cs.princeton.edu/people/profile/rpa,NaN
2,Andrew Appel,Eugene Higgins Professor,Carnegie-Mellon University,1985.0,"Software verification, computer security, prog...",NaN,NaN,https://www.cs.princeton.edu/~appel,https://www.cs.princeton.edu/sites/all/modules...,(609) 258-4627,appel@cs.princeton.edu,NaN,209 Computer Science,Computer Science,Princeton Univ,https://www.cs.princeton.edu/people/profile/appel,NaN
3,Sanjeev Arora,Charles C. Fitzmorris Professor,"University of California, Berkeley",1994.0,Uses of randomness in complexity theory and al...,NaN,NaN,https://www.cs.princeton.edu/~arora,https://www.cs.princeton.edu/sites/all/modules...,(609) 258-3869,arora@cs.princeton.edu,NaN,407 Computer Science,Computer Science,Princeton Univ,https://www.cs.princeton.edu/people/profile/arora,NaN
4,David August,Professor,"University of Illinois, Urbana/Champaign",2000.0,Computer Architecture and Compilers,NaN,NaN,https://www.cs.princeton.edu/~august,https://www.cs.princeton.edu/sites/all/modules...,(609) 258-2085,august@cs.princeton.edu,NaN,221 Computer Science,Computer Science,Princeton Univ,https://www.cs.princeton.edu/people/profile/au...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Olga Troyanskaya,Professor,Stanford University,2003.0,Bioinformatics; analysis of large-scale biolog...,NaN,NaN,/people/profile/ogt,https://www.cs.princeton.edu/sites/all/modules...,(609) 258-1749,ogt@cs.princeton.edu,NaN,320 Computer Science,Computer Science,Princeton Univ,https://www.cs.princeton.edu/people/profile/ogt,NaN
64,David Walker,"Professor, Director of Undergraduate Studies",Cornell University,2001.0,"Programming languages, type systems, compilers...",NaN,NaN,https://www.cs.princeton.edu/~dpw,https://www.cs.princeton.edu/sites/all/modules...,(609) 258-7654,dpw@cs.princeton.edu,NaN,211 Computer Science,Computer Science,Princeton Univ,https://www.cs.princeton.edu/people/profile/dpw,NaN
65,Kevin Wayne,Phillip Y. Goldman '86 University Lecturer,Cornell University,1999.0,Algorithms and data structures; computer scien...,NaN,NaN,https://www.cs.princeton.edu/~wayne,https://www.cs.princeton.edu/sites/all/modules...,(609) 258-4455,wayne@cs.princeton.edu,NaN,040 Corwin Hall,Computer Science,Princeton Univ,https://www.cs.princeton.edu/people/profile/wayne,NaN
66,Matthew Weinberg,Associate Professor,Massachusetts Institute of Technology,2014.0,NaN,NaN,NaN,https://www.cs.princeton.edu/~smattw,https://www.cs.princeton.edu/sites/all/modules...,(609) 258-0944,smattw@cs.princeton.edu,NaN,317 Computer Science,Computer Science,Princeton Univ,https://www.cs.princeton.edu/people/profile/sm...,NaN


In [7]:
names = df["name"].to_list()

In [8]:
len(names), names 

(68,
 ['Parastoo Abtahi',
  'Ryan Adams',
  'Andrew Appel',
  'Sanjeev Arora',
  'David August',
  'Lennart Beringer',
  'Suma Bhat',
  'Jack Brassil',
  'Mark Braverman',
  'Sebastian Caldas',
  'Bernard Chazelle',
  'Danqi Chen',
  "Marcel Dall'Agnol",
  'Jia Deng',
  'Adji Bousso Dieng',
  'Robert Dondero',
  'Zeev Dvir\n(on leave)',
  'Benjamin Eysenbach',
  'Christiane Fellbaum',
  'Adam Finkelstein',
  'Robert Fish',
  'Ruth Fong',
  'Michael Freedman',
  'Donna Gabai',
  'Tom Griffiths',
  'Aarti Gupta',
  'Elad Hazan',
  'Felix Heide',
  'Kyle Jamieson\n(on leave)',
  'Alan Kaplan',
  'Brian Kernighan',
  'Zachary Kincaid',
  'Gillat Kol',
  'Aleksandra Korolova',
  'Amit Levy',
  'Kai Li',
  'Xiaoyan Li',
  'Edo Liberty',
  'Wyatt Lloyd',
  'Alex Lombardi',
  'Margaret Martonosi',
  'Jonathan Mayer',
  'Andrés Monroy-Hernández',
  'Christopher Moretti',
  'Radhika Nagpal',
  'Karthik Narasimhan',
  'Arvind Narayanan',
  'Ravi Netravali',
  'Rotem Oshman',
  'Pedro Paredes',
  

## fetch google scholar data for all CS faculty

In [9]:
ntest = -1  #  2 # 

In [10]:
pub_data = []
for n, name in enumerate(names[:ntest]):
#     if n < 5: continue 
    print(f"n, name = {n}, {name} ...")
    
    author_org = f"{name} {org2}"
    norm_auth_org = normalize_str(author_org)
    file_author = f"data/GScholar_{norm_auth_org}.json"

    try:
        search_query = scholarly.search_author(author_org)
        init_result = next(search_query)
    except Exception as ex:
        print(str(ex))
        init_result = None 
        
    if init_result is None:
        print(f"Failed search_author()")
        continue

    scholar_id = init_result.get("scholar_id", "")
    if not scholar_id:
        print(f"Missing scholar_id")
        continue        
    
    # fetch data
    author = scholarly.fill(init_result)
    
    author_dict = {}
    # fill data cell
    author_dict["name"] = name 
    author_dict["file_author"] = file_author 
    author_dict["scholar_id"] = scholar_id 
    author_dict["affiliation"] = author.get("affiliation", "")
    author_dict["interests"] = "; ".join(author.get("interests", []))
    author_dict["url_author"] = get_scholar_page(scholar_id)
    author_dict["url_picture"] = author.get("url_picture", "")
    author_dict["url_homepage"] = author.get("homepage", "")
    author_dict["citedby"] = author.get("citedby", 0)
    author_dict["citedby5y"] = author.get("citedby5y", 0)
    author_dict["hindex"] = author.get("hindex", 0)
    author_dict["hindex5y"] = author.get("hindex5y", 0)
    author_dict["i10index"] = author.get("i10index", 0)
    author_dict["i10index5y"] = author.get("i10index5y", 0)
    author_dict["num_papers"] = len(author.get("publications", []))
    author_dict["num_coauthors"] = len(author.get("coauthors", []))
      
    # fill row 
    author_data = []
    for c in SCHOLAR_HEADER:
        author_data.append(author_dict.get(c))
        
    # accumulate row
    pub_data.append(author_data)
    
    # persist author data
    with open(Path(file_author), "w", encoding="utf-8") as f:
        f.write(json.dumps(author))
    
    delay = randint(1,5)
    sleep(delay)

n, name = 0, Parastoo Abtahi ...
n, name = 1, Ryan Adams ...
n, name = 2, Andrew Appel ...
n, name = 3, Sanjeev Arora ...
n, name = 4, David August ...
n, name = 5, Lennart Beringer ...
n, name = 6, Suma Bhat ...

Failed search_author()
n, name = 7, Jack Brassil ...

Failed search_author()
n, name = 8, Mark Braverman ...

Failed search_author()
n, name = 9, Sebastian Caldas ...

Failed search_author()
n, name = 10, Bernard Chazelle ...
n, name = 11, Danqi Chen ...
n, name = 12, Marcel Dall'Agnol ...

Failed search_author()
n, name = 13, Jia Deng ...
n, name = 14, Adji Bousso Dieng ...
n, name = 15, Robert Dondero ...

Failed search_author()
n, name = 16, Zeev Dvir
(on leave) ...

Failed search_author()
n, name = 17, Benjamin Eysenbach ...

Failed search_author()
n, name = 18, Christiane Fellbaum ...

Failed search_author()
n, name = 19, Adam Finkelstein ...
n, name = 20, Robert Fish ...
n, name = 21, Ruth Fong ...
n, name = 22, Michael Freedman ...
n, name = 23, Donna Gabai ...

Failed

In [11]:
len(pub_data)

48

## write out xlsx

In [12]:
df_out = pd.DataFrame(pub_data, columns=SCHOLAR_HEADER)

In [13]:
df_out.head()

,name,affiliation,interests,num_papers,num_coauthors,citedby,hindex,i10index,citedby5y,hindex5y,i10index5y,scholar_id,url_author,url_picture,url_homepage,file_author
0,Parastoo Abtahi,"Assistant Professor of Computer Science, Princ...",Human-Computer Interaction; Augmented Reality;...,14,16,490,9,7,490,9,7,u3eIIOsAAAAJ,https://scholar.google.com/citations?user=u3eI...,https://scholar.google.com/citations?view_op=m...,http://www.parastooabtahi.com/,data/GScholar_parastoo_abtahi_princeton.json
1,Ryan Adams,Princeton University,Machine Learning; Artificial Intelligence; Sta...,217,36,36687,61,130,31187,55,114,grQ_GBgAAAAJ,https://scholar.google.com/citations?user=grQ_...,https://scholar.google.com/citations?view_op=m...,http://www.cs.princeton.edu/~rpa/,data/GScholar_ryan_adams_princeton.json
2,Andrew Appel,"Professor of Computer Science, Princeton Unive...",,211,44,17797,59,132,3695,29,63,wC_ntLYAAAAJ,https://scholar.google.com/citations?user=wC_n...,https://scholar.google.com/citations?view_op=m...,http://www.cs.princeton.edu/~appel/,data/GScholar_andrew_appel_princeton.json
3,Sanjeev Arora,"Professor of Computer Science, Princeton Unive...",theoretical machine learning; theoretical comp...,250,90,33565,73,136,17012,51,107,RUP4S68AAAAJ,https://scholar.google.com/citations?user=RUP4...,https://scholar.google.com/citations?view_op=m...,http://www.cs.princeton.edu/~arora/,data/GScholar_sanjeev_arora_princeton.json
4,David August,"Computer Science, Princeton University",Compilers and Computer Architecture,170,48,8364,46,90,1852,25,48,Wov5tYoAAAAJ,https://scholar.google.com/citations?user=Wov5...,https://scholar.google.com/citations?view_op=m...,,data/GScholar_david_august_princeton.json


In [14]:
# import xlsxwriter
file_xlsx = f"data/cs-faculty-gscholar-{org2}-{n}.xlsx"
writer = pd.ExcelWriter(Path(file_xlsx), engine='xlsxwriter')
df_out.to_excel(writer, sheet_name=org2, index=False)
writer.save()

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_6604\3066051369.py:5: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [15]:
file_xlsx

'data/cs-faculty-gscholar-Princeton-66.xlsx'